# Elementary Statistical Analysis and Visualization of Video Game Data

### In this notebook I aim to grasp a better understanding of visual analysis in python by cutting my teeth on video game data scraped from the web. First I will scrape data from an online video games database. I will then munge and parse the data for use, and finally visualize several aspects of the data using python code in order to extract a variety of conclusions. While the end conclusions I reach from the data are not the overall goal of this project, I hope to come away with at least a few worthwhile ideas about games I may want to put on my "to-play" list.

## Section 0: Resources and Acknowledgements
### I will be consulting a number of websites and textbooks for this project. I will list them below as I use them, both as credit where it's due and to give anyone who may read this a good list of resources should they embark on a similar learning endeavor:

1: As always when I need a git refresher: Pro Git by Scott Chacon and Ben Straub - available at https://git-scm.com/book/en/v2

2: Twitch IGDB API Documentation - available at https://api-docs.igdb.com/#about

  2a: Twitch IGDB API Python Documentation - available at https://github.com/twitchtv/igdb-api-python


## Section 1: Scraping the Data

### I will be using the IGDB.com games database to scrape data for these analyses. The documentation for the IGDB API can be found in Secion 0.

In [4]:
#typically I would list all of my imports in one section, but as this is largely a learning exercise I will import packages in the sections at which they become relevant.
import numpy as np 
import pandas as pd 
from igdb.wrapper import IGDBWrapper
import requests
import json


#read credentials auth.json from project directory - this is user info and is hterefore kept private - auth.json is in .gitignore
user_info =  json.loads(open('auth.json').read())
user_info['grant_type']='client_credentials'


r = requests.post('https://id.twitch.tv/oauth2/token', params=user_info)
access_token = json.loads(r._content)['access_token']
expires_in = json.loads(r._content)['expires_in']
wrapper = IGDBWrapper(user_info['client_id'], access_token)

In [9]:
#scape platform data


platforms = wrapper.api_request(
    'platforms',
    'fields *; limit 500;'

)

platforms = platforms.decode("utf-8")
platforms_json = json.loads(platforms)

platforms_df = pd.DataFrame(platforms_json)

platforms_df
print(platforms_df.shape)





(171, 16)
